In [50]:
!pip3 install names
!pip3 install rstr


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [42]:
import random
import pandas as pd
import numpy as np

import names
import rstr

In [43]:
# defaults 
NO_OF_ROWS = 500
FILENAME = "simulated_survey_data.csv"

In [44]:
# NRIC generator
def generate_nric(): 
    return rstr.xeger(r'^[STFG]\d{7}[A-Z]$')

In [45]:
"""
|---------- |-----------------|---------------|--------|-----------|
| NRIC      | researcher_name | sector        | salary | degree    | 
|---------- |------------------------------------------------------|
| T8270500P | Ted Osborn      | Agricultural  | 6000   | Masters   |
| S6754779S | Edith Ward      | Aerospace     | 5000   | Bachelors |
| G4976879R | Eric Heard      | Engineering   | 7000   | Masters   |
| S4190526H | Franklin Brod   | Healthcare    | 9000   | PhD       |
|------------------------------------------------------------------|

Types:
NRIC: string 
Sector: string (categorical)
Researcher: string 
Salary: int 
Degree: int (categorical)
"""

SECTOR_TYPES = ["Agricultural", "Aerospace", "Engineering", "Healthcare"] 
QUALIFICATION_TYPES = ["PhD", "Masters", "Bachelors", "Non-degree", "Post-grad"]

# NATIONALITY_TYPES = ["SC", "PR", "FC"]

def create_simulated_data(no_of_records=10000):
    simulated_data = pd.DataFrame()
    
    # NRIC
    simulated_data["NRIC"] = [generate_nric() for _ in range(0, no_of_records)]
    
    # researcher_name
    simulated_data["researcher_name"] = [names.get_full_name() for _ in range(0, no_of_records)]
    
    # sector
    simulated_data["sector"] = np.random.choice(SECTOR_TYPES, size=no_of_records) 
    
    # salary
    # using gamma distribution 
    # https://en.wikipedia.org/wiki/Gamma_distribution
    simulated_data["salary"] = np.random.randint(4000, 12000, size=no_of_records)
    # simulated_data["salary"] = np.round(np.random.gamma(10, 300, size=no_of_records), 2)

    # degree
    simulated_data["degree"] = np.random.choice(QUALIFICATION_TYPES, size=no_of_records) 
     
    return simulated_data

In [46]:
simulated_data = create_simulated_data(no_of_records=1000)
simulated_data

,NRIC,researcher_name,sector,salary,degree
0,T4632965N,Nicole Kallman,Engineering,5016,Bachelors
1,T9804006O,Nancy Gonzales,Healthcare,11830,Post-grad
2,T2934239H,Joseph Rigby,Engineering,8520,Masters
3,S6248584N,Michelle Sheriff,Healthcare,4354,Bachelors
4,G8665162J,Lester Odonnell,Healthcare,6871,Bachelors
...,...,...,...,...,...
995,F1162281O,Vivian Williams,Healthcare,7305,Masters
996,S6773428X,Donna Robinson,Agricultural,10451,Masters
997,G9054199M,Melissa Prestridge,Engineering,8307,Non-degree
998,T8662462H,Carol Wilson,Aerospace,11248,Bachelors


In [51]:
simulated_data.to_csv("data/simulated_data.csv", mode='w', index=False)